In [6]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
sys.path.append(PROJECT_ROOT)

In [7]:
#loading data in dataframe

import pandas as pd

resume_clean_df = pd.read_csv("../Data/processed/resume_clean.csv")
jobs_clean_df = pd.read_csv("../Data/processed/jobs_clean.csv")

In [3]:
# filter NaN before sampling:
resume_clean_df = resume_clean_df.dropna(subset=["clean_resume_text", "Category"])
jobs_clean_df = jobs_clean_df.dropna(subset=["clean_job_text", "title"])

In [3]:
resume_clean_df.columns

Index(['ID', 'Resume_str', 'Category', 'clean_resume_text'], dtype='object')

In [4]:
jobs_clean_df.columns

Index(['job_id', 'title', 'description', 'skills_desc',
       'formatted_experience_level', 'formatted_work_type', 'clean_job_text'],
      dtype='object')

In [ ]:
#Testing ats_match_score function

from src.matching.ats_matcher import ats_match_score

sample_resume = resume_clean_df.sample(1)

resume = sample_resume["clean_resume_text"].values[0]
category = sample_resume["Category"].values[0]

sample_job = jobs_clean_df.sample(1)

job = sample_job["clean_job_text"].values[0]
job_title = sample_job["title"].values[0]

ats_match_score(resume, job)



np.float32(39.83)

In [11]:
resume_clean_df.iloc[0]

ID                                                            16852973
Resume_str                    HR ADMINISTRATOR/MARKETING ASSOCIATE\...
Category                                                            HR
clean_resume_text    hr administrator marketing associate hr admini...
Name: 0, dtype: object

In [12]:
resume_clean_df = resume_clean_df.loc[:, ~resume_clean_df.columns.str.contains('^Unnamed')]

In [13]:
resume_clean_df.columns

Index(['ID', 'Resume_str', 'Category', 'clean_resume_text'], dtype='object')

In [15]:
print(type(resume))
print(len(resume))

<class 'str'>
5027


In [ ]:
# saving cell should be removed after phase 1 cleaning but i am putting it in comment

#resume_clean_df = resume_clean_df.rename(
#    columns={"clean_resume": "clean_resume_text"})

In [ ]:
# NOTE:
# This saving step was required in Phase 1 (data cleaning).
# In later phases we only load the cleaned CSVs, so this cell
# can be skipped or removed in production.


# CODE 1 
#jobs_clean_df.to_csv("../Data/processed/jobs_clean.csv", index=False)
#resume_clean_df.to_csv("../Data/processed/resume_clean.csv", index=False)

#correct saving code?, as per gpt. As the above one was making errors as 'Data' was written before as 'data' and creating new csv in notebooks

# CODE 2
# import os

#PROJECT_ROOT = os.path.abspath("..")
#PROCESSED_PATH = os.path.join(PROJECT_ROOT, "Data", "processed")

#jobs_clean_df.to_csv(os.path.join(PROCESSED_PATH, "jobs_clean.csv"), index=False)
#resume_clean_df.to_csv(os.path.join(PROCESSED_PATH, "resume_clean.csv"), index=False)

In [19]:
# This gives random combinations and better validation.

sample_resume = resume_clean_df.sample(1)
sample_job = jobs_clean_df.sample(1)

resume = sample_resume["clean_resume_text"].values[0]
category = sample_resume["Category"].values[0]

job = sample_job["clean_job_text"].values[0]
job_title = sample_job["title"].values[0]


In [20]:
from src.matching.ats_matcher import final_ats_score

result = final_ats_score(
    resume_text=resume,
    job_text=job,
    job_title=job_title,
    category=category
)

print(result)
print("Final Score:", result["final_score"])
print("Confidence:", result["confidence"])
print("Domain:", result["domain"])
print("Matched Skills:", result["matched_skills"])
print("Missing Skills:", result["missing_skills"])


Domain: Design
Skill Source: domain fallback
Skills Used: ['photoshop', 'illustrator', 'figma', 'ui ux']
Matched Skills: ['photoshop', 'illustrator']
{'final_score': 29.18, 'semantic_score': 27.4, 'skill_score': 50.0, 'title_score': 0, 'confidence': 'Low Match', 'domain': 'Design', 'matched_skills': ['photoshop', 'illustrator'], 'missing_skills': ['figma', 'ui ux']}
Final Score: 29.18
Confidence: Low Match
Domain: Design
Matched Skills: ['photoshop', 'illustrator']
Missing Skills: ['figma', 'ui ux']


In [5]:
#kept for analysis not scoring

from src.utils.domain_mapper import map_domain

resume_clean_df["Domain"] = resume_clean_df["Category"].apply(map_domain)
resume_clean_df["Domain"].value_counts()



Domain
Operations              580
Other                   460
Finance                 351
Design                  307
Sales                   235
Marketing               206
IT                      120
Software Development    118
HR                      110
Name: count, dtype: int64

<h2>Tests before phase 2.2 completion 

In [8]:
#test1 data , test before making changes in stable ats system and adding more advanced features
resume = "Data Scientist with experience in Python, machine learning, statistics, pandas, and numpy.Built predictive models and performed data analysis using scikit-learn."
job = "We are hiring a Data Scientist.Required skills: Python, machine learning, statistics, pandas, numpy.Experience in predictive modeling preferred."
job_title = "Data Scientist"
category ="Data Science"

In [16]:
#test2 data , test before making changes in stable ats system and adding more advanced features
resume = "Graphic designer skilled in Photoshop, Illustrator, branding, and logo design.Experience creating marketing posters and brochures."
job = "We are hiring a Data Scientist.Required skills: Python, machine learning, statistics, pandas."
job_title = "Data Scientist"
category = "Designer"

In [13]:
#test3 data , test before making changes in stable ats system and adding more advanced features
resume = "Python machine learning statistics pandas numpy scikit-learn matplotlib seaborn SQL deep learning NLP regression classification clustering"
job = "Looking for Data Scientist with skills in Python, machine learning, statistics, pandas."
job_title = "Data Scientist"
category = "data science"

In [17]:
from src.matching.ats_matcher import final_ats_score

result = final_ats_score(
    resume_text=resume,
    job_text=job,
    job_title=job_title,
    category=category
)

print(result)
print("Final Score:", result["final_score"])
print("Confidence:", result["confidence"])
print("Domain:", result["domain"])
print("Matched Skills:", result["matched_skills"])
print("Missing Skills:", result["missing_skills"])


Domain: Design
Skill Source: domain fallback
Skills Used: ['photoshop', 'illustrator', 'figma', 'ui ux']
Matched Skills: ['photoshop', 'illustrator']
{'final_score': 37.47, 'semantic_score': 39.24, 'skill_score': 50.0, 'title_score': 0, 'confidence': 'Low Match', 'domain': 'Design', 'matched_skills': ['photoshop', 'illustrator'], 'missing_skills': ['figma', 'ui ux']}
Final Score: 37.47
Confidence: Low Match
Domain: Design
Matched Skills: ['photoshop', 'illustrator']
Missing Skills: ['figma', 'ui ux']


In [ ]:
#saving test outputs

#import json

#with open("../outputs/examples/example_strong_match.json", "w") as f:
#    json.dump(result, f, indent=4)

In [ ]:
#import json

#with open("../outputs/examples/example_moderate_match.json", "w") as f:
#    json.dump(result, f, indent=4)

In [ ]:
#import json

#with open("../outputs/examples/example_low_match.json", "w") as f:
#    json.dump(result, f, indent=4)